In [1]:
import openpyxl
import os
import json
import psycopg2
from config import config
from db_service import addrecordsdb

In [2]:
def to_camel_case(input_string):
    words = input_string.split()

    camel_case_words = [words[0].lower()] + [word.capitalize() for word in words[1:]]

    return ''.join(camel_case_words)

In [3]:
def header_dict_util(sheet_obj):
    
    in_group = False
    header_row = None
    data_row = None
    file_name_lst = []
    header_dict = {}


    for row in sheet_obj.iter_rows(values_only=True):
        if any(row):  
            if not in_group:
                header_row = None
                data_row = None
                in_group = True
            if header_row is None:
                header_row = row
            elif data_row is None:
                data_row = row
                result_dict = dict(zip(header_row, data_row))
                file_name_lst.append(result_dict)
        elif in_group:
            in_group = False

    for i, result_dict in enumerate(file_name_lst, start=1):

        temp = {"title":result_dict["formTitle"], 
                "subTitle":result_dict["assetClassName"],
                "asset_class_code":result_dict["assetClassCode"],
                "asset_class_name":result_dict["assetClassName"],
                "plan_name":result_dict["planName"],
                "pm_title":result_dict["pmItem"].lower()
               }
        header_dict[f'Group_{i}'] = temp
    
    return(header_dict)
    

In [4]:
def body_dict_util(sheet_obj):
    
    in_group = False
    current_group = None
    file_counter = 0
    body_components=[]
    body_components_dict={}
    start_column = 1
    end_column = 8 


    for row in sheet_obj.iter_rows(values_only=True):
                
        row_data = row[start_column - 1:end_column]

        if any(row_data):

            temp = {
                "type": "container",
                "key": "",
                "label": "",
                "title": ""
                }

            if not in_group:

                in_group = True
                file_counter += 1

            if row_data[0] not in ['issueTitle','testTitle', 'tooltip', 'formTitle', 'assetClassName', 'assetClassCode', 'planName', 'pmItem']:
                
                
                
                temp['key'] = to_camel_case(row_data[1])
                temp['label'] = row_data[1]
                temp['title'] = row_data[0]
                
                print(f'Group_{i} >> ', temp)
                
                body_components.append(temp)
                

        elif in_group:

            in_group = False
            body_components_dict[f'Group_{file_counter}'] = body_components
            body_components = []

    if body_components:
        body_components_dict[f'Group_{file_counter}'] = body_components

    return body_components_dict
        

In [5]:
def db_data(header_dict, body_components_dict, json_schema, company_id, db_table):
    
    for i in header_dict:
        
        json_schema['components'][0]['title'] = header_dict[i]['title']
        json_schema['components'][0]['subTitle'] = header_dict[i]['subTitle']
        json_schema['components'][1]['components'] = body_components_dict[i]
        
        container_show_condition = "test.value"
        for container_index in range(1, 5):
            json_schema['container'][container_index]['show'][0]['condition'] = container_show_condition
        
        new_json = json.dumps(json_schema)
        
        header_dict[i]['form_json'] = new_json
        header_dict[i]['company_id'] = company_id      
        
        addrecordsdb(db_table, header_dict[i])


In [6]:
ref_json_file_path = '../src/ref_json.json'
company_id='22a8170a-f97c-432b-976a-28c4d7abf3ca'
db_table='PMItemMasterForms'

with open(ref_json_file_path, 'rb') as json_file:
    json_schema = json.load(json_file)

src_file_path = "/home/ld-164/configs-v2 (3).xlsx"
save_dir = "../sheets/"
exp = ['ch.25']

In [8]:
wb_obj = openpyxl.load_workbook(src_file_path)

for i in wb_obj.sheetnames:
        
    if "ch." in i and i not in exp:
        
        print('ch >>> ', i)

        sheet_obj = wb_obj[i]
        header_dict = header_dict_util(sheet_obj)
        body_components_dict = body_dict_util(sheet_obj)
        
        db_data(header_dict, body_components_dict, json_schema, company_id, db_table)


ch >>>  ch.12
Group_1 >>  {'title': 'NFPA 70B® Enhanced De-Energized Visual Inspection Form', 'subTitle': 'Switchgear (<= 1000V)', 'asset_class_code': 'SWGR-LV', 'asset_class_name': 'Switchgear (<= 1000V)', 'plan_name': '70B-ENHANCED', 'pm_title': 'visualinspection'}
Group_2 >>  {'title': 'NFPA 70B® Enhanced De-Energized Visual Inspection Form', 'subTitle': 'Switchgear (> 1000V)', 'asset_class_code': 'SWGR-MV', 'asset_class_name': 'Switchgear (> 1000V)', 'plan_name': '70B-ENHANCED', 'pm_title': 'visualinspection'}
Group_3 >>  {'title': 'NFPA 70B® Enhanced De-Energized Visual Inspection Form', 'subTitle': 'Unitized Substation (USS) (<= 1000V)', 'asset_class_code': 'USSX-LV', 'asset_class_name': 'Unitized Substation (USS) (<= 1000V)', 'plan_name': '70B-ENHANCED', 'pm_title': 'visualinspection'}
Group_4 >>  {'title': 'NFPA 70B® Enhanced De-Energized Visual Inspection Form', 'subTitle': 'Unitized Substation (USS) (> 1000V)', 'asset_class_code': 'USSX-MV', 'asset_class_name': 'Unitized Subs

83beebd8-d871-4dda-808c-d04433de3100
<class 'uuid.UUID'>
83beebd8-d871-4dda-808c-d04433de3100
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
1846c81f-b28b-4799-9eaa-5c6de11e94a2
<class 'uuid.UUID'>
1846c81f-b28b-4799-9eaa-5c6de11e94a2
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
09c80284-a2ce-4c01-b935-676f50f212e8
<class 'uuid.UUID'>
09c80284-a2ce-4c01-b935-676f50f212e8
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
bb951dae-3b96-4bf6-bbd3-bd8b5a2117c3
<class 'uuid.UUID'>
bb951dae-3b96-4bf6-bbd3-bd8b5a2117c3
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
86f2fadf-aa76-43eb-accb-88d0fb381152
<class 'uuid.UUID'>
86f2fadf-aa76-43eb-accb-88d0fb381152
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
5d073f11-7ec3-44e7-b408-50453829faf9
<class 'uuid.UUID'>
5d073f11-7ec3-44e7-b408-50453829faf9
<class 'str'>
Databas

Database connection closed.
Connecting to the PostgreSQL database...
eee73ecf-3b18-4002-8e9f-db2beaed43b2
<class 'uuid.UUID'>
eee73ecf-3b18-4002-8e9f-db2beaed43b2
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
33ed5d55-90db-47ff-b927-6a8577524f2a
<class 'uuid.UUID'>
33ed5d55-90db-47ff-b927-6a8577524f2a
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
14e2ca07-aa3d-4a6b-a90a-4c45ff630a34
<class 'uuid.UUID'>
14e2ca07-aa3d-4a6b-a90a-4c45ff630a34
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
2ec132df-9bd0-439d-8fd3-99712981e20d
<class 'uuid.UUID'>
2ec132df-9bd0-439d-8fd3-99712981e20d
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
c3ad1106-6977-425a-95d5-07c4c37b8a85
<class 'uuid.UUID'>
c3ad1106-6977-425a-95d5-07c4c37b8a85
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
06b51566-5247-4a1d-935b-18d289cc39bb
<class 'u

74f4e470-4e52-45a3-a1f1-e927e0506b78
<class 'uuid.UUID'>
74f4e470-4e52-45a3-a1f1-e927e0506b78
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
66d8d6e4-9c9f-421f-9a9f-e2cab960c5cb
<class 'uuid.UUID'>
66d8d6e4-9c9f-421f-9a9f-e2cab960c5cb
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
b3cc777b-aed7-4dc5-b2e4-f9a700e35ef8
<class 'uuid.UUID'>
b3cc777b-aed7-4dc5-b2e4-f9a700e35ef8
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
edad4eed-95d6-429e-a352-8c739dde1fde
<class 'uuid.UUID'>
edad4eed-95d6-429e-a352-8c739dde1fde
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
4c2ce87b-1984-4156-b31a-100b6e3dc1df
<class 'uuid.UUID'>
4c2ce87b-1984-4156-b31a-100b6e3dc1df
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
2b48b35c-09bd-4e5a-83ad-fab5b3b17b0e
<class 'uuid.UUID'>
2b48b35c-09bd-4e5a-83ad-fab5b3b17b0e
<class 'str'>
Databas

90934aec-b983-4839-b552-6324390ce0a8
<class 'uuid.UUID'>
90934aec-b983-4839-b552-6324390ce0a8
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
45f470e4-a151-4a44-b7a3-c720555f7288
<class 'uuid.UUID'>
45f470e4-a151-4a44-b7a3-c720555f7288
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
65605f10-e627-4f73-974f-e9fcffd8a6ec
<class 'uuid.UUID'>
65605f10-e627-4f73-974f-e9fcffd8a6ec
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
443b33ca-73fd-44b0-a24c-008db0f3ba3d
<class 'uuid.UUID'>
443b33ca-73fd-44b0-a24c-008db0f3ba3d
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
9ba50ff8-b79f-491f-bd27-1c8f9698e5d4
<class 'uuid.UUID'>
9ba50ff8-b79f-491f-bd27-1c8f9698e5d4
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
99add414-34aa-46fd-b04a-2cf27286a719
<class 'uuid.UUID'>
99add414-34aa-46fd-b04a-2cf27286a719
<class 'str'>
Databas

Database connection closed.
Connecting to the PostgreSQL database...
e56f7272-6761-414a-831d-1b88ea361f1d
<class 'uuid.UUID'>
e56f7272-6761-414a-831d-1b88ea361f1d
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
0d80d90b-33e6-4071-a1a2-1ff5512f2bca
<class 'uuid.UUID'>
0d80d90b-33e6-4071-a1a2-1ff5512f2bca
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
2f1ca857-07e5-4612-b04d-031d5c9a1a6e
<class 'uuid.UUID'>
2f1ca857-07e5-4612-b04d-031d5c9a1a6e
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
308c32d1-6da1-4511-b8cd-8d297be05a1d
<class 'uuid.UUID'>
308c32d1-6da1-4511-b8cd-8d297be05a1d
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
cd2dbab1-74a0-4e94-a2b5-7f3ba6d34e4b
<class 'uuid.UUID'>
cd2dbab1-74a0-4e94-a2b5-7f3ba6d34e4b
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
4f6a865f-26e4-41ae-8135-c7962c9a6600
<class 'u

f1d24615-64ca-4437-9b49-294803fb835f
<class 'uuid.UUID'>
f1d24615-64ca-4437-9b49-294803fb835f
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
18795897-b392-4a17-96c7-b506b3872ee6
<class 'uuid.UUID'>
18795897-b392-4a17-96c7-b506b3872ee6
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
d56adfb2-5462-4abc-89a0-7f123de61a21
<class 'uuid.UUID'>
d56adfb2-5462-4abc-89a0-7f123de61a21
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
62de2292-9db9-446d-a655-ac4e5c860954
<class 'uuid.UUID'>
62de2292-9db9-446d-a655-ac4e5c860954
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
99f103fd-66da-4b41-94d9-44c2bf2d92b6
<class 'uuid.UUID'>
99f103fd-66da-4b41-94d9-44c2bf2d92b6
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
4ccb2662-50bb-44f7-a880-0eab7889eaf1
<class 'uuid.UUID'>
4ccb2662-50bb-44f7-a880-0eab7889eaf1
<class 'str'>
Databas

Database connection closed.
Connecting to the PostgreSQL database...
76ea5193-b599-4def-b8b7-a50d0c97241f
<class 'uuid.UUID'>
76ea5193-b599-4def-b8b7-a50d0c97241f
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
01707f67-c396-489a-a3e0-2d81057f0895
<class 'uuid.UUID'>
01707f67-c396-489a-a3e0-2d81057f0895
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
8a008768-e958-4b82-a20a-ff40c0ca5bad
<class 'uuid.UUID'>
8a008768-e958-4b82-a20a-ff40c0ca5bad
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
26f8876d-c7c8-4e9e-bee6-8fb5a5d3c76f
<class 'uuid.UUID'>
26f8876d-c7c8-4e9e-bee6-8fb5a5d3c76f
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
47c3fd7f-19e0-463c-8b3f-a7636d83964a
<class 'uuid.UUID'>
47c3fd7f-19e0-463c-8b3f-a7636d83964a
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
22b85071-548f-4198-a4fa-1e93a8960b90
<class 'u

dff129bb-9eee-48b8-abd6-374ae136fb8f
<class 'uuid.UUID'>
dff129bb-9eee-48b8-abd6-374ae136fb8f
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
4e49bd7e-6e6f-48aa-aa56-dc9ad390c3d3
<class 'uuid.UUID'>
4e49bd7e-6e6f-48aa-aa56-dc9ad390c3d3
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
10f55e88-6471-4d3e-b9c1-ec20b8c1c861
<class 'uuid.UUID'>
10f55e88-6471-4d3e-b9c1-ec20b8c1c861
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
375f738f-5e10-4a0d-a288-d2919d1b0446
<class 'uuid.UUID'>
375f738f-5e10-4a0d-a288-d2919d1b0446
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
01f737f4-eeb3-4662-b7f1-ca96a51916d8
<class 'uuid.UUID'>
01f737f4-eeb3-4662-b7f1-ca96a51916d8
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
7280b3bc-0062-44f0-81c6-b67ce86057d2
<class 'uuid.UUID'>
7280b3bc-0062-44f0-81c6-b67ce86057d2
<class 'str'>
Databas

Database connection closed.
Connecting to the PostgreSQL database...
1f6dccba-e311-457f-8a90-df1ca521e503
<class 'uuid.UUID'>
1f6dccba-e311-457f-8a90-df1ca521e503
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
adb78b31-b68f-4026-b1c3-4c69dabd4f22
<class 'uuid.UUID'>
adb78b31-b68f-4026-b1c3-4c69dabd4f22
<class 'str'>
Database connection closed.
Connecting to the PostgreSQL database...
331c8f50-a5fc-49fe-89bd-bb67fbfdc90b
<class 'uuid.UUID'>
331c8f50-a5fc-49fe-89bd-bb67fbfdc90b
<class 'str'>
Database connection closed.
